In [6]:
import torch
import datetime
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import pickle
import numpy as np
import time
import random
from collections import defaultdict
from UV_Encoders import UV_Encoder
from UV_Aggregators import UV_Aggregator
from Social_Encoders import Social_Encoder
from Social_Aggregators import Social_Aggregator
import torch.nn.functional as F
import torch.utils.data
from dataPreprocess import dataPreprocess
from model import GraphRec
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import datetime
import argparse
import os
import winsound


"""
GraphRec: Graph Neural Networks for Social Recommendation. 
Wenqi Fan, Yao Ma, Qing Li, Yuan He, Eric Zhao, Jiliang Tang, and Dawei Yin. 
In Proceedings of the 28th International Conference on World Wide Web (WWW), 2019. Preprint[https://arxiv.org/abs/1902.07243]

If you use this code, please cite our paper:
```
@inproceedings{fan2019graph,
  title={Graph Neural Networks for Social Recommendation},
  author={Fan, Wenqi and Ma, Yao and Li, Qing and He, Yuan and Zhao, Eric and Tang, Jiliang and Yin, Dawei},
  booktitle={WWW},
  year={2019}
}
```

"""
          #graph         
def train(model, device, train_loader, optimizer, epoch, best_rmse, best_mae):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        batch_nodes_u, batch_nodes_v, labels_list = data
        optimizer.zero_grad()
        loss = model.loss(batch_nodes_u.to(device), batch_nodes_v.to(device), labels_list.to(device))
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f, The best rmse/mae: %.6f / %.6f' % (
                epoch, i, running_loss / 100, best_rmse, best_mae))
            running_loss = 0.0
    return 0


def test(model, device, test_loader):
    model.eval()
    tmp_pred = []
    target = []
    with torch.no_grad():
        for test_u, test_v, tmp_target in test_loader:
            test_u, test_v, tmp_target = test_u.to(device), test_v.to(device), tmp_target.to(device)
            val_output = model.forward(test_u, test_v)
            tmp_pred.append(list(val_output.data.cpu().numpy()))
            target.append(list(tmp_target.data.cpu().numpy()))
    tmp_pred = np.array(sum(tmp_pred, []))
    target = np.array(sum(target, []))
    expected_rmse = sqrt(mean_squared_error(tmp_pred, target))
    mae = mean_absolute_error(tmp_pred, target)
#     print('tmp_pred\n',tmp_pred)
#     print('target\n',target)
    
    return expected_rmse, mae,tmp_pred,target

In [7]:
print('begin')
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
    print('True')
device = torch.device("cuda" if use_cuda else "cpu")

print('device',device)

begin
True
device cuda


In [18]:
batch_size=128
embed_dim=64
lr=0.001
test_batch_size=1000
epochs=100
data='data/Epinions'
embed_dim = embed_dim
dir_data = data
path_data = ''


In [19]:

for file in os.listdir(dir_data):
    if file.endswith(".pickle"):
        path_data = dir_data + '/' + file
        print("\nPickle File Exists, No need to generate one....\n")
if path_data == '':
    print('\ndataset.pickle not found. Generating a pickle file.......\n')
    dp = dataPreprocess(dir_data)
    print('a')
    dp.preprocess()
    print('b')
    path_data = dir_data + '/dataset.pickle'
print('\n***** Data Loaded ****\n')
       


dataset.pickle not found. Generating a pickle file.......

a
b

***** Data Loaded ****



In [20]:

            
    # data_file = open(path_data, 'rb')
    if not dir_data == 'data/Toy':
        # history_u_lists = pickle.load(data_file)
        # history_ur_lists = pickle.load(data_file)
        # history_v_lists = pickle.load(data_file)
        # history_vr_lists = pickle.load(data_file)
        # train_u = pickle.load(data_file)
        # train_v = pickle.load(data_file)
        # train_r = pickle.load(data_file)
        # test_u = pickle.load(data_file)
        # test_v = pickle.load(data_file)
        # test_r = pickle.load(data_file)
        # social_adj_lists = pickle.load(data_file)
        # ratings_list = pickle.load(data_file)
        # users = pickle.load(data_file)
        # friends = pickle.load(data_file)
        # trust = pickle.load(data_file)

        
        history_u_lists = dp.userItemDict
        history_ur_lists = dp.userRatings
        history_v_lists = dp.itemUserDict
        history_vr_lists = dp.itemRatings
        train_u = dp.trainUsers
        train_v = dp.trainItems
        train_r = dp.trainRatings
        test_u = dp.testUsers
        test_v = dp.testItems
        test_r = dp.testRatings
        social_adj_lists = dp.userUserDict
        # print('social_adj_lists',social_adj_lists)
        ratings_list = dp.ratingsL
        users =dp.user
        friends = dp.friend
        trust = dp.trust


        usersL = list(set(users+friends))
        num_users = max(max(usersL), max(train_u), max(test_u))
        num_items = max(max(list(train_v)), max(list(test_v)))
        num_ratings = ratings_list.__len__()
        
    else:
        data_file = open(path_data, 'rb')
        history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, train_u, train_v, train_r, test_u, test_v, test_r, social_adj_lists, ratings_list = pickle.load(data_file)
        num_users = history_u_lists.__len__()
        num_items = history_v_lists.__len__()
        num_ratings = ratings_list.__len__()


In [21]:
#social_adj_lists[1]


In [22]:
trainset = torch.utils.data.TensorDataset(torch.LongTensor(train_u), torch.LongTensor(train_v),
                                            torch.FloatTensor(train_r))

testset = torch.utils.data.TensorDataset(torch.LongTensor(test_u), torch.LongTensor(test_v),
                                            torch.FloatTensor(test_r))

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=True)


In [23]:

u2e = nn.Embedding(num_users, embed_dim).to(device)
v2e = nn.Embedding(num_items, embed_dim).to(device)
r2e = nn.Embedding(num_ratings, embed_dim).to(device)


first_embeddings=u2e,v2e,r2e

In [24]:
print(u2e)
print(v2e)
print(r2e)

print(len(history_u_lists))
print(len(history_v_lists))
print(len(ratings_list))

print(len(train_u))
print(len(train_v))
print(len(train_r))

print(len(test_u))
print(len(test_v))
print(len(test_r))

150432


Embedding(49289, 64)
Embedding(139738, 64)
Embedding(5, 64)
37365
113937
5
465376
465376
465376
199448
199448
199448


150432

In [25]:
agg_u_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=True)
enc_u_history = UV_Encoder(u2e, embed_dim, history_u_lists, history_ur_lists, agg_u_history, cuda=device, uv=True)

agg_u_social = Social_Aggregator(lambda nodes: enc_u_history(nodes).t(), u2e, embed_dim, cuda=device)
enc_u = Social_Encoder(lambda nodes: enc_u_history(nodes).t(), embed_dim, social_adj_lists, agg_u_social,
                        base_model=enc_u_history, cuda=device)
agg_v_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=False)
enc_v_history = UV_Encoder(v2e, embed_dim, history_v_lists, history_vr_lists, agg_v_history, cuda=device, uv=False)


In [26]:
print(u2e.weight[1][1])
print(v2e.weight[1][1])
print(r2e.weight[1][1])

tensor(0.7183, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-1.6210, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(1.5562, device='cuda:0', grad_fn=<SelectBackward0>)


In [27]:


# model
graphrec = GraphRec(enc_u, enc_v_history, r2e).to(device)
optimizer = torch.optim.RMSprop(graphrec.parameters(), lr=lr, alpha=0.9)




best_embeddings=u2e,v2e,r2e


best_rmse = 9999.0
best_mae = 9999.0
endure_count = 0
RR=range(1, epochs + 1)
Length=len(RR)
epoch=1

In [31]:
# ALPHA=1 # DIFFERANCE CONDITION
# BETA=1  # MODIFY TRUST CONDITION

In [32]:
print('# DIFFERANCE CONDITION ',ALPHA)
print('# MODIFY TRUST CONDITION ',BETA)

for epoch in RR:
    print('epoch :',epoch,'/',Length,' : ',datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print('train')
    train(graphrec, device, train_loader, optimizer, epoch, best_rmse, best_mae)
    

    
    print('test')
    expected_rmse, mae ,tmp_pred,target= test(graphrec, device, test_loader)
    # please add the validation set to tune the hyper-parameters based on your datasets.
    # early stopping (no validation set in toy dataset)
    if best_rmse > expected_rmse:
        best_rmse = expected_rmse
        best_mae = mae
        endure_count = 0
        
        winsound.Beep(440,1000)
    else:
        endure_count += 1

    print("rmse: %.4f, mae:%.4f " % (expected_rmse, mae))

    
    
#     if endure_count > 5:
#         filehandler = open('best_embeddings(u2e-v2e-r2e)', 'w') 
#         filehandler = open('', 'w') 
#         pickle.dump(best_embeddings, filehandler)
#         filehandler.close()
#         break



# DIFFERANCE CONDITION  1
# MODIFY TRUST CONDITION  1
epoch : 1 / 100  :  2022-05-29 18:48:15
train


KeyError: 34830

In [580]:
history_u_lists[1] , history_ur_lists[1]

([34, 60, 102, 0, 52, 46, 33, 45, 145, 6, 68],
 [3, 2, 4, 4, 2, 3, 3, 4, 4, 5, 2])

In [581]:
type(history_u_lists)

collections.defaultdict

In [582]:
usersRating=[]
# usersRatingEncoded=[]

In [583]:

for i in range(len(history_u_lists)):
    usersRating.append([])
#     usersRatingEncoded.append([])
    for j in range(len(history_u_lists[i])):
        usersRating[i].append((history_u_lists[i][j],history_ur_lists[i][j]))
#         usersRatingEncoded[i].append((history_u_lists[i][j]*8+history_ur_lists[i][j]))
#         print([i,history_u_lists[i][j],history_ur_lists[i][j]])
#         print(history_u_lists[i][j]*8+history_ur_lists[i][j])
#     usersRatingEncoded[i].sort()
        


In [584]:
print(usersRating[704],usersRating[3])
# print(usersRatingEncoded[704],usersRatingEncoded[3])


[(102, 3), (55, 3), (80, 3), (74, 3), (62, 3), (31, 3)] [(203, 3), (52, 5), (33, 2), (24, 3)]


In [585]:
itemsRating=[]

In [586]:

for i in range(len(history_v_lists)):
    itemsRating.append([])
    for j in range(len(history_v_lists[i])):
        itemsRating[i].append((history_v_lists[i][j],history_vr_lists[i][j]))
#         print([i,history_v_lists[i][j],history_vr_lists[i][j]])


In [587]:
itemsRating[2]

[(172, 2), (193, 7), (37, 2), (53, 4), (292, 4)]

In [588]:
print(290,social_adj_lists[290])
print(497,social_adj_lists[497])


290 {289, 497}
497 {290, 388}


In [589]:
usersRating[290],usersRating[497]

([(11, 3), (8, 5), (47, 3), (45, 3), (0, 5), (13, 2)],
 [(57, 5), (76, 2), (31, 3), (24, 5), (62, 2)])

In [590]:
# item * 8 + id_de_rating
0*8+0,0*8+7,1*8+0,1*8+7

(0, 7, 8, 15)

In [591]:
def get_key(my_dict,val):     
    for key, value in my_dict.items():          
        if val == value:              
            return key       
    return "There is no such Key"     

In [592]:

print(social_adj_lists[1])
l=set()
l.add(1)

def removeItemFromSetIfExist(mySet,itemToRemove):
    if itemToRemove in mySet:
        mySet.remove(itemToRemove)


{0}


In [593]:
def addTrustBetween(social_adj_lists, i,j):
    if i not in social_adj_lists[j]:
        print(i,j,' added')
        social_adj_lists[j].add(i)
        social_adj_lists[i].add(j)
    else:
#         print(i,j,' already exist!')
        pass
    
def removeTrustBetween(social_adj_lists, i,j):
    if i not in social_adj_lists[j]:
#         print(i,j,' already do not Exist')
        pass
    else:
        print(i,j,' REMOVED!')
        removeItemFromSetIfExist(social_adj_lists[i],j)
        removeItemFromSetIfExist(social_adj_lists[j],i)
    

def addTrustDegreeBetween(i,j,trustDegree):
    if i>j:
        k=i
        i=j
        j=k
#     if (i,j) == (53,146):
#         print(53,146)
    if str((i,j)) not in trustDegree:
#         print('not')
        trustDegree[str((i,j))]=[]
        trustDegree[str((i,j))].append(1)
        trustDegree[str((i,j))].append((i,j))
    else:
#         print('in')
        trustDegree[str((i,j))][0]=trustDegree[str((i,j))][0]+1
    
def addUnTrustDegreeBetween(i,j,UnTrustDegree):
    if i>j:
        k=i
        i=j
        j=k
#     if (i,j) == (53,146):
#         print(53,146)
    if str((i,j)) not in UnTrustDegree:
#         print('not')
        UnTrustDegree[str((i,j))]=[]
        UnTrustDegree[str((i,j))].append(1)
        UnTrustDegree[str((i,j))].append((i,j))
        
    else:
#         print('in')
        UnTrustDegree[str((i,j))][0]=UnTrustDegree[str((i,j))][0]+1
    



In [594]:
usersRating[0]

[(55, 0)]

In [595]:

ALPHA=5 # DIFFERANCE CONDITION

trustDegree={}
print('# finding trust similarity')
for user,userRating in enumerate(usersRating):
#     if user in [53,146]:
#         print(user)
    for i,couple_item_rating in enumerate(userRating):
        # print(i,couple_item_rating)
        
        item=couple_item_rating[0]
        rating=couple_item_rating[1]
        #print('\t',itemsRating[item])
        for user_, rating_ in itemsRating[item]:
#             print(item,rating_,rating)

#             if (user_,user) == (53,146):
#                 print(53,146)
            if user!=user_ and abs(rating-rating_) < ALPHA:
#                 print(user,rating)
                addTrustDegreeBetween(user,user_,trustDegree)

    

UnTrustDegree={}
print('# finding untrust unsimilarity')
for user,userRating in enumerate(usersRating):
#     if user in [53,146]:
#         print(user)
    for i,couple_item_rating in enumerate(userRating):
        # print(i,couple_item_rating)
        
        item=couple_item_rating[0]
        rating=couple_item_rating[1]
        #print('\t',itemsRating[item])
        for user_, rating_ in itemsRating[item]:
#             print(item,rating_,rating)

#             if (user_,user) == (53,146):
#                 print(53,146)
            if user!=user_ and abs(rating-rating_) >ALPHA:
#                 print(user,rating)
                addUnTrustDegreeBetween(user,user_,UnTrustDegree)

# finding trust similarity
# finding untrust unsimilarity


In [596]:
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################

In [597]:
UnTrustDegree[str((53, 146))]

[4, (53, 146)]

In [598]:
trustDegree[str((53, 146))]

[136, (53, 146)]

In [599]:
len(trustDegree)

186108

In [600]:
max(trustDegree.values())

[136, (53, 146)]

In [601]:
print(max(trustDegree.values()),get_key(trustDegree,max(trustDegree.values()))) 

[136, (53, 146)] (53, 146)


In [602]:
social_adj_lists[53],social_adj_lists[146],

({51, 146, 147, 148, 250, 251, 252, 253, 254, 474, 501, 659},
 {53, 147, 148, 149})

In [603]:
len(usersRating[53]),len(usersRating[146]),

(211, 162)

In [604]:
social_adj_lists[1]

{0}

In [605]:
social_adj_lists[161]

{162, 163}

In [606]:
BETA=14

print('# ADD TRUST  BY CONSIDERING SIMILARITY')
for key,tr in trustDegree.items():
#     print(key,tr)
    degreeSimilarity=-1
    degreeUnsimilarity=-1
    couple=tr[1]
    degreeSimilarity=tr[0]
    if(str(tr[1])in UnTrustDegree):
        degreeUnsimilarity=UnTrustDegree[str(tr[1])][0]
        if degreeSimilarity < BETA *degreeUnsimilarity:
            continue
#     if degreeUnsimilarity == -1:
#         print(degreeSimilarity,degreeUnsimilarity)
#         pass
#     else:
#         print(degreeSimilarity,degreeUnsimilarity)
#         print(couple[0],couple[1])
#         if degreeSimilarity > len
#         pass
    if degreeSimilarity < BETA:
        continue
#     if degreeSimilarity < BETA:
#             print('ERROR')
#             break
    addTrustBetween(social_adj_lists,couple[0],couple[1])


print('# REMOVE TRUST  BY CONSIDERING UNSIMILARITY')
for key,tr in UnTrustDegree.items():
#     print(key,tr)
    degreeSimilarity=-1
    degreeUnsimilarity=-1
    couple=tr[1]
    degreeUnsimilarity=tr[0]
    if(str(couple) in trustDegree):
        degreeSimilarity=trustDegree[str(couple)][0]
        if degreeUnsimilarity < BETA * degreeSimilarity:
            continue
#     if degreeUnsimilarity == -1:
#         print(degreeSimilarity,degreeUnsimilarity)
#         pass
#     else:
#         print(degreeSimilarity,degreeUnsimilarity)
#         print(couple[0],couple[1])
#         if degreeSimilarity > len
#         pass
    if degreeUnsimilarity < BETA:
        continue
#     if degreeUnsimilarity < BETA:
#             print('ERROR')
#             break
    removeTrustBetween(social_adj_lists,couple[0],couple[1])

    

# ADD TRUST  BY CONSIDERING SIMILARITY
1 258  added
1 164  added
1 348  added
1 227  added
1 529  added
1 639  added
1 212  added
1 457  added
1 51  added
1 126  added
1 444  added
1 644  added
1 234  added
1 343  added
1 515  added
1 328  added
1 89  added
1 48  added
1 157  added
1 195  added
1 39  added
1 608  added
1 366  added
1 339  added
1 276  added
1 252  added
1 374  added
1 424  added
1 528  added
1 487  added
1 464  added
1 93  added
1 668  added
1 695  added
1 535  added
1 300  added
1 401  added
1 696  added
1 360  added
1 325  added
1 199  added
1 546  added
1 525  added
1 161  added
1 228  added
1 516  added
1 485  added
1 113  added
1 242  added
1 479  added
1 701  added
1 204  added
1 562  added
1 660  added
1 231  added
1 359  added
1 159  added
1 648  added
1 384  added
1 336  added
1 498  added
1 637  added
1 504  added
1 298  added
1 540  added
1 54  added
1 257  added
1 549  added
1 703  added
1 73  added
1 700  added
1 590  added
1 656  added
1 666  added
1 218 

12 549  added
12 343  added
12 105  added
12 485  added
12 464  added
13 619  added
13 363  added
13 262  added
13 347  added
13 693  added
13 225  added
13 611  added
13 493  added
13 499  added
13 483  added
13 299  added
13 189  added
13 151  added
13 51  added
13 231  added
13 521  added
13 674  added
13 644  added
13 199  added
13 485  added
13 93  added
13 668  added
13 681  added
13 296  added
13 185  added
13 503  added
13 614  added
13 528  added
13 701  added
13 325  added
13 454  added
13 277  added
13 256  added
13 83  added
13 366  added
13 695  added
13 227  added
13 216  added
13 637  added
13 374  added
13 384  added
13 428  added
13 298  added
13 200  added
13 356  added
13 70  added
13 89  added
13 460  added
13 126  added
13 603  added
13 694  added
13 340  added
13 549  added
13 426  added
13 39  added
13 236  added
13 457  added
13 536  added
13 660  added
13 156  added
13 161  added
13 696  added
13 646  added
13 173  added
13 669  added
13 498  added
13 360  adde

33 639  added
33 118  added
33 360  added
33 264  added
33 242  added
33 359  added
33 89  added
33 483  added
33 54  added
33 212  added
33 348  added
33 366  added
33 70  added
33 374  added
33 186  added
33 499  added
33 105  added
33 93  added
33 336  added
33 420  added
33 631  added
33 73  added
33 384  added
33 159  added
33 700  added
33 252  added
33 694  added
34 340  added
34 98  added
34 409  added
34 318  added
34 135  added
34 499  added
34 673  added
34 366  added
34 226  added
34 700  added
34 325  added
34 338  added
34 48  added
34 375  added
34 146  added
34 485  added
34 212  added
34 348  added
34 646  added
34 399  added
34 426  added
34 693  added
34 249  added
34 204  added
34 259  added
34 458  added
34 649  added
34 42  added
34 105  added
34 189  added
34 268  added
34 257  added
34 252  added
34 281  added
34 464  added
34 666  added
34 516  added
34 643  added
34 353  added
34 36  added
34 43  added
34 479  added
34 424  added
34 78  added
34 593  added
34 

42 74  added
42 208  added
43 619  added
43 214  added
43 524  added
43 292  added
43 268  added
43 681  added
43 253  added
43 88  added
43 53  added
43 146  added
43 459  added
43 256  added
43 315  added
43 377  added
43 366  added
43 460  added
43 365  added
43 104  added
43 329  added
43 87  added
43 248  added
43 126  added
43 90  added
43 198  added
43 468  added
43 699  added
43 96  added
43 113  added
43 204  added
43 91  added
43 75  added
43 458  added
43 135  added
43 197  added
43 124  added
43 636  added
43 503  added
43 650  added
43 401  added
43 289  added
43 228  added
43 109  added
43 231  added
43 456  added
43 328  added
43 242  added
43 358  added
43 479  added
43 391  added
43 646  added
43 529  added
43 428  added
43 551  added
43 193  added
43 666  added
43 447  added
43 498  added
43 486  added
43 259  added
43 424  added
43 213  added
43 119  added
43 628  added
43 375  added
43 52  added
43 673  added
43 338  added
43 190  added
43 485  added
43 700  added
4

51 90  added
51 423  added
51 161  added
51 503  added
51 479  added
51 639  added
51 509  added
51 600  added
51 632  added
51 542  added
51 466  added
51 89  added
51 593  added
51 669  added
51 396  added
51 384  added
51 515  added
51 95  added
51 694  added
51 328  added
51 132  added
51 197  added
51 257  added
51 674  added
51 160  added
51 152  added
51 666  added
51 356  added
51 252  added
51 420  added
51 374  added
51 187  added
51 445  added
51 419  added
51 449  added
51 529  added
51 104  added
51 486  added
51 543  added
51 499  added
51 577  added
51 521  added
51 588  added
51 199  added
51 185  added
51 665  added
51 264  added
51 528  added
51 325  added
51 366  added
51 695  added
51 556  added
51 637  added
51 512  added
51 379  added
51 70  added
51 689  added
51 401  added
51 692  added
51 318  added
51 603  added
51 638  added
51 457  added
51 660  added
51 656  added
51 546  added
51 118  added
51 525  added
51 114  added
51 608  added
51 228  added
51 550  ad

...

481 503  added
481 536  added
481 546  added
481 504  added
483 611  added
483 493  added
483 499  added
483 506  added
483 521  added
483 674  added
483 644  added
483 588  added
483 485  added
483 668  added
483 681  added
483 665  added
483 503  added
483 614  added
483 528  added
483 695  added
483 556  added
483 637  added
483 677  added
483 619  added
483 689  added
483 603  added
483 694  added
483 638  added
483 660  added
483 669  added
483 498  added
483 591  added
483 643  added
483 516  added
483 550  added
483 631  added
483 502  added
483 520  added
483 494  added
483 593  added
483 666  added
483 486  added
483 533  added
483 543  added
483 529  added
483 700  added
483 636  added
483 536  added
483 527  added
483 626  added
483 544  added
483 531  added
483 683  added
483 667  added
483 639  added
483 487  added
483 616  added
483 535  added
483 592  added
483 698  added
483 509  added
483 510  added
483 632  added
483 595  added
483 606  added
483 585  added
483 641  a

503 600  added
503 611  added
503 534  added
503 674  added
503 588  added
503 681  added
503 665  added
503 614  added
503 619  added
503 689  added
503 603  added
503 638  added
503 536  added
503 660  added
503 617  added
503 542  added
503 696  added
503 591  added
503 525  added
503 557  added
503 643  added
503 608  added
503 550  added
503 587  added
503 551  added
503 648  added
503 508  added
503 636  added
503 650  added
503 616  added
503 558  added
503 539  added
503 667  added
503 510  added
503 612  added
503 512  added
503 641  added
503 570  added
503 585  added
503 632  added
503 522  added
503 693  added
503 683  added
503 628  added
503 626  added
503 533  added
503 697  added
503 543  added
503 642  added
503 698  added
503 544  added
503 509  added
503 606  added
503 625  added
503 526  added
504 521  added
504 588  added
504 540  added
504 701  added
504 637  added
504 544  added
504 628  added
504 698  added
504 528  added
504 646  added
504 529  added
504 667  a

528 610  added
528 668  added
528 644  added
528 568  added
528 579  added
528 703  added
528 699  added
528 590  added
528 593  added
528 637  added
528 639  added
528 643  added
528 632  added
528 617  added
528 562  added
528 656  added
528 666  added
528 660  added
528 698  added
528 588  added
528 550  added
528 631  added
528 674  added
528 700  added
528 701  added
528 585  added
528 549  added
528 572  added
528 552  added
528 693  added
528 665  added
528 600  added
528 622  added
528 533  added
528 695  added
528 621  added
528 543  added
528 680  added
528 603  added
528 614  added
528 694  added
528 612  added
528 673  added
528 536  added
528 542  added
528 595  added
528 636  added
528 544  added
528 628  added
528 667  added
528 650  added
528 681  added
528 692  added
528 551  added
528 683  added
529 543  added
529 694  added
529 646  added
529 666  added
529 552  added
529 669  added
529 590  added
529 639  added
529 695  added
529 579  added
529 535  added
529 540  a

558 701  added
558 590  added
558 600  added
558 668  added
558 700  added
558 699  added
558 680  added
558 644  added
558 669  added
558 593  added
558 666  added
558 694  added
562 632  added
562 656  added
562 637  added
562 699  added
562 669  added
562 593  added
562 666  added
562 660  added
562 588  added
562 590  added
562 644  added
562 631  added
562 680  added
562 643  added
562 674  added
562 700  added
562 668  added
562 701  added
562 585  added
562 693  added
562 665  added
562 703  added
562 600  added
562 695  added
562 621  added
562 639  added
562 614  added
562 694  added
562 612  added
562 673  added
562 608  added
562 591  added
562 628  added
562 646  added
562 568  added
562 611  added
562 681  added
562 683  added
562 579  added
564 568  added
564 639  added
564 681  added
564 637  added
564 693  added
564 703  added
564 611  added
564 680  added
564 585  added
564 644  added
568 593  added
568 668  added
568 652  added
568 595  added
568 636  added
568 699  a

631 696  added
631 646  added
631 700  added
631 639  added
631 637  added
631 655  added
631 699  added
631 644  added
631 673  added
631 649  added
631 701  added
631 680  added
631 666  added
631 660  added
631 632  added
631 703  added
631 693  added
631 669  added
631 641  added
631 650  added
631 694  added
631 643  added
631 642  added
631 668  added
631 681  added
631 648  added
631 674  added
631 656  added
631 695  added
631 636  added
631 665  added
632 649  added
632 642  added
632 637  added
632 699  added
632 669  added
632 666  added
632 660  added
632 644  added
632 680  added
632 641  added
632 696  added
632 643  added
632 674  added
632 700  added
632 668  added
632 701  added
632 693  added
632 703  added
632 639  added
632 697  added
632 673  added
632 694  added
632 689  added
632 646  added
632 681  added
632 650  added
632 648  added
632 667  added
632 695  added
635 700  added
636 668  added
636 699  added
636 694  added
636 639  added
636 674  added
636 700  a

In [607]:
# for key,tr in UnTrustDegree.items():
#     print(key,tr)

In [608]:
iii=0
jjj=0
for item,rat in usersRating[53]:
    for u,rat_ in itemsRating[item]:
        if u == 146 and abs(rat-rat_)<3:
            iii=iii+1
#             print(abs(rat-rat_))
        if u == 146 and abs(rat-rat_)>3:
            jjj=jjj+1
print(iii,jjj)

47 12


In [609]:
itemRating=itemsRating[1]

In [610]:
itemRating

[(81, 1), (274, 7), (184, 4), (619, 7)]

In [611]:
l={}
l[(str((1,23)))]=1

l[(str((1,23)))]
l.pop((str((1,23))))
l[(str((1,23)))]



KeyError: '(1, 23)'

In [196]:
if str((1,2)) not in l:
    print('not')
else:
    print('in')
    

not


In [198]:
l[str((1,2))]=1


In [203]:
l[str((1,2))]=l[str((1,2))]+1
l[str((1,2))]


6

In [204]:
type(l)

dict

In [55]:
my_dict ={"John":1, "Michael":2, "Shawn":3} 
def get_key(my_dict,val):     
    for key, value in my_dict.items():          
        if val == value:              
            return key       
    return "There is no such Key"     
print(get_key(my_dict,1)) 
print(get_key(my_dict,2)) 


John
Michael
